# 07-2 심층 신경망
##### 인공 신경망에 층을 여러 개 추가하여 패션 MNIST 데이터셋을 분류하면서 케라스로 심층 신경망을 만드는 방법을 알아보자

- 1절에서 만들었던 인공 신경망의 성능을 더 높여보자

### 2개의 층
- 케라스 API에서 MNIST 데이터셋을 불러오자

In [1]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

2023-12-25 17:54:45.270292: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


- 픽셀값을 0 ~ 255 범위에서 0 ~ 1 사이로 변환
- 28 X 28크기의 2차원 배열을 784 크기의 1차원 배열로 변경
- train_test_split() 함수로 훈련 세트와 검증 세트를 나눔

In [2]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

- 이제 인공 신경망 모델에 층을 2개 추가
- 만들어진 모델의 대략적인 구조는 아래와 같음
![심층신경망](./dnn-367-1.jpg)
- 인공 신경망에서 입력층과 출력층 사이에 은닉층(hidden layer)이 추가됨
- 활성화 함수 : 신경망 층의 선형 방정식의 계산 값에 적용하는 함수 (소프트맥스 함수 역시 활성화 함수)
  - 출력층에 적용하는 활성화 함수는 종류가 제한되어 있음 (이진분류 : 시그모이드, 다중 분류 : 소프트맥스)
  - 은닉층의 활성화 함순는 사용이 자유로움. 시그모이드 함수와 렐루(ReLU) 함수 등을 사용
- 은닉층에 활성화 함수를 적용하는 이유
  - 선형 방정식에서 b를 치환시켜 b의 역할을 제거 :
    - $a \times 4 + 2 = b$
    - $b \times 3 - 5 = c $
    - $a \times 12 + 1 = c$
  - 신경망에서도 마찬가지임. 은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈
  - 선형 계산을 적당히 비선형으로 비틀어주어야 함
    - 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 수행하기 위함 (아래의 식과 같음)
    - $a \times 4 + 2 = b$
    - $log(b) = k$
    - $k \times 3 - 5 = c$
    - 인공 신경망을 그림으로 나타낼 때 활성화 함수를 생략하는 경우가 많음 (활성화 함수를 층에 포함되어 있다고 간주하기 떄문)
- 많이 사용하는 활성화 함수 중 하나는 4장에서 배웠던 시그모이드 함수
  - 뉴런의 출력 z 값을 0과 1 사이로 압축
- keras로 만들어보자

In [3]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

- dense1은 은닉층이고 100개의 뉴런을 가진 밀집층. 은닉층 뉴런 개수를 정하는데에는 특별한 기준이 없음. 뉴런의 적절한 개수 판단은 상당한 경험이 필요함
- 한가지 제약 사항은 출력층의 뉴런보다는 많게 만들어야 함 (클래스 10개 확률을 예측하는데 이전 은닉층의 뉴런 10개보다 적다면 정보가 부족할 것)
- dense2는 출력층. 10개의 클래스를 분류하므로 10개의 뉴런을 두었고 활성화 함수는 소프트맥스로 지정

### 심층 신경망 만들기
- 앞서 만든 dense1, dense2를 Sequential 클래스에 추가하여 심층 신경망(DNN)을 만들어보자

In [4]:
# 출력층(dense2)을 리스트의 가장 마지막에 둘 것!
model = keras.Sequential([dense1, dense2])

- 리스트는 가장 처음 등장하는 은닉층에서 자미작 출력층 순서로 나열해야 함
- 인공 신경망의 강력한 성능은 층을 추가하며 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나옴
- 케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있음

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


- 첫 줄은 모델 이름
- 이후 모델에 들어있는 층이 순서대로 나열
- 순서는 리스트에 입력된 순서로 나열
- 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력
- 출력 크기(None, 100)
  - 샘플 개수(None) -> 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행 (미니배치 경사 하강법)
  - 케라스 기본 미니배치 크기는 32개. 이 값은 fit() 메서드에 batch_size 매개변수로 바꿀 수 있음
  - 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정
  - 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원이라고 부름
  - 두번째 출력은 : 뉴런 개수(100) -> 샘플마다 784개의 픽셀값이 은닉층을 통과하며 100개의 특성으로 압축
  - 마지막으로 모델 파라미터 개수 출력(Param, 78500) : Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있음 + 뉴런마다 1개의 절편
![Param-1](./dnn-373-1.jpg)
- 두 번째 층의 출력 크기는 (None, 10) : 배치 차원은 동일하게 None이고 출력 뉴런 개수가 10개임
- 이 층의 모델 파라미터 개수는 100개의 은닉층 뉴런과 10개의 출력층 뉴런이 모두 연결되고 출력층의 뉴런마다 하나의 절편이 있기 때문에 1,010개의 모델 파라미터가 있음
![Param-2](./dnn-373-1.jpg)

- summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련 파라미터 개수가 동일하게 79,510개 -> 은닉층과 출력층의파라미터 개수를 합친 값
- 훈련되지 않는 파라미터(Non-trainable params)는 0
  - 간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있음
  - 이런 층의 파라미터 개수가 여기 나타남

### 층을 추가하는 다른 방법
- 매개변수를 추가하여 확인해보자

In [6]:
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
                          keras.layers.Dense(10, activation='softmax', name='output')], name='패션 MNIST 모델')

- 해당 코드는 추가되는 층을 한눈에 알아보는 장점이 있음
- 모델 이름과 층은 반드시 영문이어야 함
- summary() 메서드 출력에서 확인 고고

In [7]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


- 이 방법이 편리하지만 많은 충을 추가하려면 코드가 너무 길어짐
- 이럴땐 add() 함수 사용

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
# 이 방법은 한눈에 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가 가능
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               78500     
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


- 이제 모델을 훈련해보자
- compile() 메서드 설정은 1절에서 했던 것과 동일
- 5번의 에포크 동안 훈련해보자

In [13]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5704 - accuracy: 0.8051
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4129 - accuracy: 0.8524
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3766 - accuracy: 0.8627
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3546 - accuracy: 0.8704
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3360 - accuracy: 0.8783


- 훈련 세트에 대한 성능을 보면 추가된 층이 성능을 향상시켰다는 것을 알 수 있음
- 인공 신경망에 몇 개의 층을 추가하더라도 compile()메서드와 fit() 메서드의 사용은 동일 (keras API 장점)
- 이미지 분류에서 높은 성능을 낼 수 있는 활성화 함수에 대해 알아보자

### 렐루 함수
